## Figure 1C: Genotype network of 392 Omicron viruses.
#### This notebook prepares input data for Gephi for generating the genotype network.

In [13]:
import os
import copy
import pandas as pd

In [14]:
meta = pd.read_csv('../data/rawdata/nyc_omicron_metadata.csv', index_col='seqName')

In [15]:
nextclade = pd.read_csv('../data/nextclade_nyc/nextclade.tsv', delimiter='\t', index_col='seqName')

In [16]:
muts = {}
seqs = {}

for i in nextclade.index:
    seqs[i] = []
    for s in nextclade.loc[i, 'substitutions'].split(','):
        if 266 <= int(s[1:-1]) <= 29674:
            muts[s] = muts.get(s, 0) + 1
            seqs[i].append(s)

conserved = []
for i in muts:
    if muts[i] == len(nextclade):
        conserved.append(i)
conserved.append('G22813T')
conserved.append('T23075C')

for i in conserved:
    muts.pop(i)
    for j in seqs:
        if i in seqs[j]:
            seqs[j].remove(i)

muts = dict(sorted(muts.items(), key=lambda x: int(x[0][1:-1])))


In [17]:
types = []
for i in seqs:
    if seqs[i] not in types:
        types.append(seqs[i])
types.sort(key=lambda x:len(x))

divergence_threshold = 1
links = []
networks = []
networks.append([types[0]])
for i in types[1:]:
    linked = []
    for n in networks:
        for j in n:
            if len(set(j)^set(i)) <= divergence_threshold:
                if i == []:
                    links.append(['Ancestry', '|'.join(j)])
                elif j == []:
                    links.append(['|'.join(i), 'Ancestry'])
                else:
                    links.append(['|'.join(i), '|'.join(j)])
                if networks.index(n) not in linked:
                    linked.append(networks.index(n))    
    if len(linked) == 0:
        networks.append([i])
    elif len(linked) == 1:
        networks[linked[0]].append(i)
    else:
        n_merged = [i]
        to_remove = []
        for n in linked:
            n_merged.extend(networks[n])
            to_remove.append(networks[n])
        for r in to_remove:
            networks.remove(r)
        networks.append(n_merged)

while len(networks) != 1:
    divergence_threshold += 1
    n_linked = []
    for n1 in networks:
        for i in n1:
            for n2 in networks:
                if n2 != n1:
                    for j in n2:
                        if len(set(j)^set(i)) <= divergence_threshold:
                            if i == []:
                                links.append(['Ancestry', '|'.join(j)])
                            elif j == []:
                                links.append(['|'.join(i), 'Ancestry'])
                            else:
                                links.append(['|'.join(i), '|'.join(j)])
                            n_linked.append([networks.index(n1), networks.index(n2)])

    n_connected = []
    n_connected.append(n_linked[0])
    for n in n_linked[1:]:
        linked = []
        for n_c in n_connected:
            if len(set(n)&set(n_c)) != 0:
                linked.append(n_connected.index(n_c))

        if len(linked) == 0:
            n_connected.append(n)
        elif len(linked) == 1:
            n_connected[linked[0]].extend(list(set(n)-set(n_connected[linked[0]])))
        else:
            n_merged = copy.deepcopy(n)
            to_remove = []
            for n_c in linked:
                n_merged.extend(list(set(n_connected[n_c])-set(n_merged)))
                to_remove.append(n_connected[n_c])
            for r in to_remove:
                n_connected.remove(r)
            n_connected.append(n_merged)

    merged = []
    temp_networks = []
    for i in n_connected:
        n_t = []
        for j in i:
            n_t.extend(networks[j])
            merged.append(j)
        temp_networks.append(n_t)
    for i, n in enumerate(networks):
        if i not in merged:
            temp_networks.append(n)
    networks = temp_networks

for i in links:
    i.sort()

record = []
with open('../data/genotype_network_input.csv', 'w') as f:
    f.write('Source'+','+'Target'+'\n')
    for i in links:
        if i not in record:
            f.write(','.join(i)+'\n')
            record.append(i)


In [18]:
import math

types_counts = {}
for t in types:
    types_counts['|'.join(t)] = 0
    for i in seqs:
        if seqs[i] == t:
            types_counts['|'.join(t)] += 1
for t in types_counts:
    types_counts[t] = math.log2(types_counts[t])

types_counts = dict(sorted(types_counts.items(), key=lambda x:x[1], reverse=True))
types_counts

{'G5515T': 7.348728154231077,
 '': 4.954196310386875,
 'G5515T|C16596T': 4.906890595608519,
 'G2398A|G5515T': 4.087462841250339,
 'G5924A|C23664T': 4.0,
 'T10135C|C25708T|C25886T|A29301G': 3.321928094887362,
 'G5515T|C5974T': 2.321928094887362,
 'C2470T|G22599A': 2.321928094887362,
 'C2470T|A14940G|G22599A': 2.321928094887362,
 'A4034G': 2.0,
 'G5515T|C16178T': 2.0,
 'C3990T|G5924A|T26497C': 2.0,
 'T10135C|C25708T|A29301G': 2.0,
 'C29632T': 1.584962500721156,
 'C2902T': 1.584962500721156,
 'G5515T|C8293T': 1.584962500721156,
 'C2902T|T24448C': 1.584962500721156,
 'C9110T|A29170T|C29284T': 1.584962500721156,
 'C12068T': 1.0,
 'C2706T|G5515T': 1.0,
 'G5515T|C5974T|C18115T': 1.0,
 'G5924A|C23664T|A25966G': 1.0,
 'C1710T|C2902T|T3331A|G28003T': 1.0,
 'T10135C|G14268A|C25708T|A29301G': 1.0,
 'C5025T|T10135C|C25708T|A29301G': 1.0,
 'T10135C|G14268A|A21222G|C25708T|A29301G': 1.0,
 'C12400T': 0.0,
 'C28054T': 0.0,
 'A5828G': 0.0,
 'G29651A': 0.0,
 'C2523T': 0.0,
 'C16887T': 0.0,
 'G5515T|G2423